In [1]:
# Imports
import pandas as pd

In [2]:
df = pd.read_csv('../data/raw/games.csv')
print(df.dtypes)
print("{} instances, {} dimensions".format(df.shape[0], df.shape[1]))
print(df['GAME_ID'].nunique())

GAME_DATE_EST        object
GAME_ID               int64
GAME_STATUS_TEXT     object
HOME_TEAM_ID          int64
VISITOR_TEAM_ID       int64
SEASON                int64
TEAM_ID_home          int64
PTS_home            float64
FG_PCT_home         float64
FT_PCT_home         float64
FG3_PCT_home        float64
AST_home            float64
REB_home            float64
TEAM_ID_away          int64
PTS_away            float64
FG_PCT_away         float64
FT_PCT_away         float64
FG3_PCT_away        float64
AST_away            float64
REB_away            float64
HOME_TEAM_WINS        int64
dtype: object
26651 instances, 21 dimensions
26622


We see that GAME_ID is an (almost) unique identifier. We drop the duplicate values and set it as the index. 

In [3]:
df = df.drop_duplicates(subset='GAME_ID') # drop conflicting game IDs, assign as index
df.set_index('GAME_ID', inplace=True)
print(df.shape)

(26622, 20)


Now, we want to get each team's streak data coming into the game in question, because this is what we will use to predict game outcomes. But in order to do so, we need to identify 

 remake the original dataset into a 'mirror' dataset, where each sample is replicated from the perspective of the home team and the away team. 

First, we map the original index to a new indexing system, unique for home team's and away team's perspective. 

In [4]:
sorted_index = df.index.sort_values()
print("Original index:\n {}\n".format(sorted_index))

home_index = sorted_index * 2
away_index = sorted_index * 2 + 1

print("New index, for games from home team perspective:\n {}\n".format(home_index))
print("New index, for games from away team perspective:\n {}\n".format(away_index))

Original index:
 Index([10300001, 10300002, 10300003, 10300004, 10300005, 10300006, 10300007,
       10300008, 10300009, 10300010,
       ...
       52000121, 52000131, 52000201, 52000211, 52100101, 52100111, 52100121,
       52100131, 52100201, 52100211],
      dtype='int64', name='GAME_ID', length=26622)

New index, for games from home team perspective:
 Index([ 20600002,  20600004,  20600006,  20600008,  20600010,  20600012,
        20600014,  20600016,  20600018,  20600020,
       ...
       104000242, 104000262, 104000402, 104000422, 104200202, 104200222,
       104200242, 104200262, 104200402, 104200422],
      dtype='int64', name='GAME_ID', length=26622)

New index, for games from away team perspective:
 Index([ 20600003,  20600005,  20600007,  20600009,  20600011,  20600013,
        20600015,  20600017,  20600019,  20600021,
       ...
       104000243, 104000263, 104000403, 104000423, 104200203, 104200223,
       104200243, 104200263, 104200403, 104200423],
      dtype='int64'

Next we can merge these two indices to get a fresh set of indexes for this mirror data.

In [5]:
mirror_index = home_index.union(away_index).sort_values()
print("New index format\n {}\n".format(mirror_index))

New index format
 Index([ 20600002,  20600003,  20600004,  20600005,  20600006,  20600007,
        20600008,  20600009,  20600010,  20600011,
       ...
       104200222, 104200223, 104200242, 104200243, 104200262, 104200263,
       104200402, 104200403, 104200422, 104200423],
      dtype='int64', name='GAME_ID', length=53244)



Next, we need to create the correct columns. Let's revisit what features the original data has:

In [6]:
df.columns

Index(['GAME_DATE_EST', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID',
       'SEASON', 'TEAM_ID_home', 'PTS_home', 'FG_PCT_home', 'FT_PCT_home',
       'FG3_PCT_home', 'AST_home', 'REB_home', 'TEAM_ID_away', 'PTS_away',
       'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away',
       'HOME_TEAM_WINS'],
      dtype='object')

Using the above features, we can design new features for our mirror data:

In [7]:
stat_cols = ['PTS', 'FG_PCT', 'FG3_PCT', 'AST', 'REB']
mirror_cols = ['GAME_DATE_EST', 'TEAM_ID', 'SEASON', 
                   'TEAM_ID', 'TEAM_WINS']
    
for col in stat_cols:
    mirror_cols.append("{}_for".format(col))
    mirror_cols.append("{}_against".format(col))

mirror_cols

['GAME_DATE_EST',
 'TEAM_ID',
 'SEASON',
 'TEAM_ID',
 'TEAM_WINS',
 'PTS_for',
 'PTS_against',
 'FG_PCT_for',
 'FG_PCT_against',
 'FG3_PCT_for',
 'FG3_PCT_against',
 'AST_for',
 'AST_against',
 'REB_for',
 'REB_against']

Now, we can create a new dataframe, 'mirror_df', with these indices to hold the mirror data

In [8]:
mirror_df = pd.DataFrame(columns=mirror_cols, index=mirror_index)
mirror_df.head()

,GAME_DATE_EST,TEAM_ID,SEASON,TEAM_ID,TEAM_WINS,PTS_for,PTS_against,FG_PCT_for,FG_PCT_against,FG3_PCT_for,FG3_PCT_against,AST_for,AST_against,REB_for,REB_against
GAME_ID,,,,,,,,,,,,,,,
20600002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20600003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20600004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20600005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20600006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Finally, we can begin adding data to the mirror dataframe. First we select a game instance:  

In [9]:
df.sort_index(inplace=True)
df.head()

,GAME_DATE_EST,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
GAME_ID,,,,,,,,,,,,,,,,,,,,
10300001,2003-10-05,Final,1610612762,1610612742,2003,1610612762,90.0,0.457,0.735,0.143,23.0,41.0,1610612742,85.0,0.447,0.500,0.250,20.0,38.0,1
10300002,2003-10-06,Final,1610612763,1610612749,2003,1610612763,105.0,0.494,0.618,0.267,25.0,48.0,1610612749,94.0,0.427,0.700,0.154,20.0,43.0,1
10300003,2003-10-07,Final,1610612765,1610612739,2003,1610612765,96.0,0.391,0.600,0.444,19.0,37.0,1610612739,100.0,0.494,0.690,0.667,25.0,52.0,0
10300004,2003-10-07,Final,1610612742,1610612753,2003,1610612742,99.0,0.488,0.667,0.167,20.0,52.0,1610612753,89.0,0.408,0.727,0.350,15.0,34.0,1
10300005,2003-10-07,Final,1610612757,1610612745,2003,1610612757,104.0,0.527,0.657,0.429,22.0,33.0,1610612745,80.0,0.470,0.667,0.333,10.0,37.0,1


In [10]:
id =  10300001
game = df.loc[id]
game

GAME_DATE_EST       2003-10-05
GAME_STATUS_TEXT         Final
HOME_TEAM_ID        1610612762
VISITOR_TEAM_ID     1610612742
SEASON                    2003
TEAM_ID_home        1610612762
PTS_home                  90.0
FG_PCT_home              0.457
FT_PCT_home              0.735
FG3_PCT_home             0.143
AST_home                  23.0
REB_home                  41.0
TEAM_ID_away        1610612742
PTS_away                  85.0
FG_PCT_away              0.447
FT_PCT_away                0.5
FG3_PCT_away              0.25
AST_away                  20.0
REB_away                  38.0
HOME_TEAM_WINS               1
Name: 10300001, dtype: object

Now we can manipulate the selected game, and add it to mirror_df. Remember, we want to add each data point TWICE: once from the perspective of the home team, and again from the perspective of the away team. We'll do the home team first. Let's define the new mirror game instance, and move everything over.

In [11]:
mirror_game_home = {col: None for col in mirror_cols}
mirror_game_home['TEAM_ID'] = game['HOME_TEAM_ID']
mirror_game_home['GAME_DATE_EST'] = game['GAME_DATE_EST']
mirror_game_home['SEASON'] = game['SEASON']
mirror_game_home['TEAM_WINS'] = game['HOME_TEAM_WINS']

# stat data
mirror_game_home['PTS_for'] = game['PTS_home']
mirror_game_home['PTS_against'] = game['PTS_away']
mirror_game_home['FG_PCT_for'] = game['FG_PCT_home']
mirror_game_home['FG_PCT_against'] = game['FG_PCT_away']
mirror_game_home['FG3_PCT_for'] = game['FG3_PCT_home']
mirror_game_home['FG3_PCT_against'] = game['FG3_PCT_away']
mirror_game_home['AST_for'] = game['AST_home']
mirror_game_home['AST_against'] = game['AST_away']
mirror_game_home['REB_for'] = game['REB_home']
mirror_game_home['REB_against'] = game['REB_away']
mirror_game_home

{'GAME_DATE_EST': '2003-10-05',
 'TEAM_ID': 1610612762,
 'SEASON': 2003,
 'TEAM_WINS': 1,
 'PTS_for': 90.0,
 'PTS_against': 85.0,
 'FG_PCT_for': 0.457,
 'FG_PCT_against': 0.447,
 'FG3_PCT_for': 0.143,
 'FG3_PCT_against': 0.25,
 'AST_for': 23.0,
 'AST_against': 20.0,
 'REB_for': 41.0,
 'REB_against': 38.0}

We do the same thing, but now from the perspective of the AWAY team:

In [12]:
mirror_game_away = {col: None for col in mirror_cols}
mirror_game_away['TEAM_ID'] = game['VISITOR_TEAM_ID']
mirror_game_away['GAME_DATE_EST'] = game['GAME_DATE_EST']
mirror_game_away['SEASON'] = game['SEASON']
mirror_game_away['TEAM_WINS'] = 1 - game['HOME_TEAM_WINS']

# stat data
mirror_game_away['PTS_for'] = game['PTS_away']
mirror_game_away['PTS_against'] = game['PTS_home']
mirror_game_away['FG_PCT_for'] = game['FG_PCT_away']
mirror_game_away['FG_PCT_against'] = game['FG_PCT_home']
mirror_game_away['FG3_PCT_for'] = game['FG3_PCT_away']
mirror_game_away['FG3_PCT_against'] = game['FG3_PCT_home']
mirror_game_away['AST_for'] = game['AST_away']
mirror_game_away['AST_against'] = game['AST_home']
mirror_game_away['REB_for'] = game['REB_away']
mirror_game_away['REB_against'] = game['REB_home']
mirror_game_away

{'GAME_DATE_EST': '2003-10-05',
 'TEAM_ID': 1610612742,
 'SEASON': 2003,
 'TEAM_WINS': 0,
 'PTS_for': 85.0,
 'PTS_against': 90.0,
 'FG_PCT_for': 0.447,
 'FG_PCT_against': 0.457,
 'FG3_PCT_for': 0.25,
 'FG3_PCT_against': 0.143,
 'AST_for': 20.0,
 'AST_against': 23.0,
 'REB_for': 38.0,
 'REB_against': 41.0}

Looking side by side, to see how values are inversed from perspective of each team:

In [13]:
print(mirror_game_home)
print(mirror_game_away)

{'GAME_DATE_EST': '2003-10-05', 'TEAM_ID': 1610612762, 'SEASON': 2003, 'TEAM_WINS': 1, 'PTS_for': 90.0, 'PTS_against': 85.0, 'FG_PCT_for': 0.457, 'FG_PCT_against': 0.447, 'FG3_PCT_for': 0.143, 'FG3_PCT_against': 0.25, 'AST_for': 23.0, 'AST_against': 20.0, 'REB_for': 41.0, 'REB_against': 38.0}
{'GAME_DATE_EST': '2003-10-05', 'TEAM_ID': 1610612742, 'SEASON': 2003, 'TEAM_WINS': 0, 'PTS_for': 85.0, 'PTS_against': 90.0, 'FG_PCT_for': 0.447, 'FG_PCT_against': 0.457, 'FG3_PCT_for': 0.25, 'FG3_PCT_against': 0.143, 'AST_for': 20.0, 'AST_against': 23.0, 'REB_for': 38.0, 'REB_against': 41.0}


Finally, we can tuck these data instances into our mirror_df using our index mappings:

In [14]:
home_team_id = id * 2
away_team_id = id * 2 + 1
mirror_df.loc[home_team_id] = mirror_game_home
mirror_df.loc[away_team_id] = mirror_game_away
mirror_df.head()

,GAME_DATE_EST,TEAM_ID,SEASON,TEAM_ID,TEAM_WINS,PTS_for,PTS_against,FG_PCT_for,FG_PCT_against,FG3_PCT_for,FG3_PCT_against,AST_for,AST_against,REB_for,REB_against
GAME_ID,,,,,,,,,,,,,,,
20600002,2003-10-05,1610612762,2003,1610612762,1,90.0,85.0,0.457,0.447,0.143,0.25,23.0,20.0,41.0,38.0
20600003,2003-10-05,1610612742,2003,1610612742,0,85.0,90.0,0.447,0.457,0.25,0.143,20.0,23.0,38.0,41.0
20600004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20600005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20600006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


This is the idea behind how mirror data was created from the original data (code in src). Using this mirror dataset, Further steps are then taken to engineer streak data; see streak_data.ipynb for more info. 